# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [1]:
%%time
import pandas as pd
import numpy as np
import datetime
import clickhouse_connect

CPU times: total: 750 ms
Wall time: 1.98 s


---

#  <span style="color:Blue">Juntar Ficheiros</span>

- Importar do Clickhouse

In [2]:
client = clickhouse_connect.get_client(host='ch.brandsandninjas.com', 
                                       port=443, 
                                       username='chninja', 
                                       password='ku43ueqnB5Q0AYb2C4FsJRTc7qX')

In [4]:
#Ler o ficheiro

dfClick = client.query_df('SELECT * FROM BaseDelta180')

dfClick = dfClick[dfClick.DATA != dfClick.DATA.unique()[0]]

- Importar ficheiros novos

In [250]:
dfNovo = pd.read_excel("D:\\B&N Dados\\Delta\\Diário\\dia_actual.xlsb") #xlsx


dfNovo = dfNovo[['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                         "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)"]]

# Renomear colunas e criar as do dia

dfNovo = dfNovo.rename(columns={"VND (D-1)": "SELLOUT_1_Dias_Antes", 
                                "SOH": "STOCK_1_Dias_Antes",
                                "INTRANSIT": "INTRANSIT_1_Dias_Antes", 
                                "EXPECTED": "EXPECTED_1_Dias_Antes", 
                                "PRES_STOCK": "PRES_STOCK_1_Dias_Antes"})

dfNovo["SELLOUT_1_Dias_Antes"] = np.where(dfNovo["SELLOUT_1_Dias_Antes"]<0, 0, dfNovo["SELLOUT_1_Dias_Antes"])
dfNovo['SELLOUT'] = dfNovo.groupby(["STORE","EAN"])['SELLOUT_1_Dias_Antes'].shift(-1)
dfNovo['STOCK'] = dfNovo.groupby(["STORE","EAN"])['STOCK_1_Dias_Antes'].shift(-1)
dfNovo['INTRANSIT'] = dfNovo.groupby(["STORE","EAN"])['INTRANSIT_1_Dias_Antes'].shift(-1)
dfNovo['EXPECTED'] = dfNovo.groupby(["STORE","EAN"])['EXPECTED_1_Dias_Antes'].shift(-1)
dfNovo['PRES_STOCK'] = dfNovo.groupby(["STORE","EAN"])['PRES_STOCK_1_Dias_Antes'].shift(-1)

#DATETIME
# se xlsx
#dfNovo['DATA'] = pd.to_datetime(dfNovo['DATA'], format='%d-%m-%Y') 

# se xlsb
dfNovo['DATA'] = pd.to_datetime(dfNovo['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

- Produtos específicos

In [251]:
# Passar produtos e lojas para listas

produtos = dfClick.DESC_ARTIGO.unique().tolist()
lojas = dfClick.STORE.unique().tolist()

# Alterar o dataframe para apenas incluir os produtos e lojas em causa

dfNovo = dfNovo[(dfNovo["DESC_ARTIGO"].isin(produtos)) & (dfNovo["STORE"].isin(lojas))].copy()



- Juntar

In [254]:
dfDiario = pd.concat([dfClick, dfNovo], ignore_index=True, join='outer')

dfFinal = dfDiario.copy()

- Vendedores

In [256]:
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

dfFinal = dfFinal.drop(columns="Vendedor")

# Criar coluna de reposição
dfFinal = pd.merge(dfFinal, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

# <font color=green>Ficheiro Lido<font>

---

# Colunas de métricas interessantes

> - ROTURA
> - PRÉ_ROTURA

In [257]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)
dfFinal["ROTURA_1_Dias_Antes"] = np.where((dfFinal["STOCK_1_Dias_Antes"] <= 0) & (dfFinal["PRES_STOCK"].shift(1) > 0), 1, 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)
dfFinal["PRE_ROTURA_1_Dias_Antes"] = (dfFinal["STOCK_1_Dias_Antes"] < dfFinal["PRES_STOCK"].shift(1)).astype(int)

# Colunas de métricas 30, 60, 120 e 180 dias antes

In [259]:
diasMetHist = [30, 60, 120, 180]

> - Volatilidade de Procura: <br>
coeficiente de variação

In [260]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Volatilidade_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Percentagem de Rotura: <br>
média de roturas $* 100$

In [261]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Roturas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100

> - Percentagem de Supply:<br>
média de vezes que foi pedido stock $*100$

In [262]:
# Sempre que é pedido abastecimento, fazer com que seja 1

dfFinal["New_Supply"] = np.where((dfFinal["EXPECTED"].shift(1)==0) & (dfFinal["EXPECTED"]>0), 1, 0)

In [263]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Supply_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Efeito fim de semana

In [264]:
#dfFinal['SELLOUT_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["SELLOUT"].copy()
#dfFinal['SELLOUT_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["SELLOUT"].copy()
dfFinal['SELLOUT_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["SELLOUT_1_Dias_Antes"].copy()
dfFinal['SELLOUT_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["SELLOUT_1_Dias_Antes"].copy()

In [265]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Efeito_Fds_{i}"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())/
                                              (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
                                               .transform(lambda x: x.rolling(window=i, min_periods=1).mean())))-1)*100


> - Tempo médio inter-supply

In [266]:
dfFinal["InterSupply"] = np.where(dfFinal["EXPECTED"]==0, 1, 0)

groups = (dfFinal['InterSupply'] != dfFinal['InterSupply'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'InterSupply': 'sum'}).reset_index(drop=True)
result = result[result['InterSupply'] > 0]

dfFinal = dfFinal.drop(columns=['InterSupply'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

In [267]:
for i in diasMetHist:
    dfFinal[f"Percentagem_InterSupplyMed_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Tempo indisponível

In [268]:
'''O que vai acontecer é: no primeiro dia em que há rotura vai aparecer a soma de todos os dias com rotura a seguir a esse!
Todos os outros valores serão NaN para não serem considerados quando for feita a média. Assim a média corresponderá ao
número médio de dias em que se deixa um produto em rotura.'''

dfFinal["Dias_Indisponivel_1_Dias_Antes"] = np.where(dfFinal["ROTURA_1_Dias_Antes"]==1, 1, 0)

In [269]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Dias_Indisponivel_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Dias_Indisponivel_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Percentagem de dias em Stock Borderline

In [270]:
dfFinal["Percentagem_Stock_Borderline_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<0.2*dfFinal["PRES_STOCK"].shift(1), 1, 0)

In [271]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Dias_Stock_Borderline_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Stock_Borderline_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Percentagem de dias de Linear Incompleto

In [272]:
dfFinal["Percentagem_Linear_Incompleto_1_Dias_Antes"] = np.where(dfFinal["STOCK_1_Dias_Antes"]<dfFinal["PRES_STOCK"].shift(1), 1, 0)

In [273]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Dias_Linear_Incompleto_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Percentagem_Linear_Incompleto_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Percentagem de dias sem vendas

In [274]:
dfFinal["Sem_Vendas_1_Dias_Antes"] = np.where(dfFinal["SELLOUT_1_Dias_Antes"] == 0, 1, 0)

In [275]:
for i in diasMetHist:
    dfFinal[f"Percentagem_Dias_Sem_Vendas_{i}"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).mean()))*100


> - Vendas perdidas

In [276]:
dfFinal['ROTURA_fds_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([5,6,0])]["ROTURA_1_Dias_Antes"].copy()
dfFinal['ROTURA_semana_1_Dias_Antes'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([1,2,3,4])]["ROTURA_1_Dias_Antes"].copy()

In [277]:
for i in diasMetHist:
    dfFinal[f"Vendas_Perdidas_em_{i}_Dias"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum())\
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).median())) \
                                                  + 
                                               (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).sum()) \
                                                  * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes'].transform(lambda x: x.rolling(window=i, min_periods=1).median())))
    

# Métricas até 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [278]:
# Quantos dias antes:

diaI=4         #dia inicial
diaF=5       #dia final

diasMet = [1, 4, 5, 10]

> Função

In [279]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores

> - SELLOUTS

In [280]:
%%time
# Usar função para sellouts até 10 dias antes

for i in diasMet:
    dias(dfFinal, i, "SELLOUT")

CPU times: total: 172 ms
Wall time: 163 ms


> - STOCKS

In [281]:
# Usar função para Stocks até 10 dias antes

for i in diasMet:
    dias(dfFinal, i, "STOCK")

> > - Ordenar

> - INTRANSIT e EXPECTED

In [282]:
# Usar função para Trânsito até 10 dias antes


for i in diasMet:
    dias(dfFinal, i, "INTRANSIT")
    
for i in diasMet:
    dias(dfFinal, i, "EXPECTED")

> - STK

In [283]:
# STK do dia = soma dos stocks em loja com os stocks em trânsito no próprio dia

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

for i in diasMet:
    dias(dfFinal, i, "STK")

> - MSA

In [284]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10, min_periods=1).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10, min_periods=1).std())

for i in diasMet:
    dias(dfFinal, i, "MSA10")

    
dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20, min_periods=1).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20, min_periods=1).std())
  
for i in diasMet:
    dias(dfFinal, i, "MSA20")

> - CICLOS

In [285]:
# Coluna de Ciclos de reposição

dfFinal["CICLOS"] = dfFinal["STOCK_1_Dias_Antes"]/dfFinal["PRES_STOCK"]

for i in diasMet:
    dias(dfFinal, i, "CICLOS")

> - Adequação de Stock

In [286]:
# Coluna de Adequacao de stock


dfFinal["Adequacao"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in diasMet:
    dias(dfFinal, i, "Adequacao")

C:\Users\joao_\AppData\Local\Temp\ipykernel_5688\4171257849.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores
C:\Users\joao_\AppData\Local\Temp\ipykernel_5688\4171257849.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores
C:\Users\joao_\AppData\Local\Temp\ipykernel_5688\4171257849.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To re

> - Smart é com semana e fim de semana

In [287]:
%%time
from datetime import datetime, timedelta


# Define a function to calculate weekday and weekend counts
def calculate_weekday_weekend_counts(date):
    weekday_count = 0
    weekend_count = 0
    
    for _ in range(5):
        if date.weekday() < 4:  # Monday to Thursday
            weekday_count += 1
        else:  # Friday to Sunday
            weekend_count += 1
        
        date += timedelta(days=1)
    
    return pd.Series({"CONTAGEM_SEMANA": weekday_count, "CONTAGEM_FIMSEMANA": weekend_count})

# Apply the function to each row in the DataFrame
dfFinal[["CONTAGEM_SEMANA", "CONTAGEM_FIMSEMANA"]] = dfFinal["DATA"].apply(calculate_weekday_weekend_counts)



# 5 dias antes
dfFinal["Balance_Smart"] = ((
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])
    +
    (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA']))
     / dfFinal["STK_1_Dias_Antes"])
dfFinal["Balance_Smart"] = np.where(dfFinal["STK"]<=0, -1, dfFinal["Balance_Smart"])
dfFinal["Balance_Smart_5_Dias_Antes"] = dfFinal["Balance_Smart"].shift(4)

CPU times: total: 9.66 s
Wall time: 9.67 s


In [288]:
dfFinal["RISCO"] = np.where((dfFinal.Balance_Smart > 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.2) & (dfFinal.Percentagem_Roturas_120 > 5), 1,
                   np.where((dfFinal.Balance_Smart > 1) & (dfFinal.Balance_Smart < 2) & (dfFinal.CICLOS_5_Dias_Antes < 0.6) & (dfFinal.Percentagem_Roturas_120 < 3), 2, 3))

In [289]:
dfFinal["RISCO_ROTURA"] = np.where((dfFinal.Balance_Smart >= 12.5) | (dfFinal.Balance_Smart < 0), 1, 
                          np.where((dfFinal.Balance_Smart >= 1.1) & (dfFinal.Balance_Smart < 12.5), 2,
                          np.where((dfFinal.Balance_Smart >= 0.833) & (dfFinal.Balance_Smart < 1.1), 3,
                          np.where((dfFinal.Balance_Smart >= 0.087) & (dfFinal.Balance_Smart < 0.833), 4, 5))))         

> - Dias para rotura de Stock

In [290]:
dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STOCK_1_Dias_Antes"] / (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])+(dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA'])

dfFinal["Dias_para_Rotura_Stock_5_Dias_Antes"] = dfFinal["Dias_para_Rotura_Stock"].shift(4)

> - Dias para rotura de Linear

In [291]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duracao_Linear'] = dfFinal["PRES_STOCK"] / (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_FIMSEMANA'])+(dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana_1_Dias_Antes']\
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())* dfFinal['CONTAGEM_SEMANA'])

dfFinal["Dias_Duracao_Linear_5_Dias_Antes"] = dfFinal["Dias_Duracao_Linear"].shift(4)

# <font color=blue>Escrever

- Dias certos

In [292]:
# Ficheiro Dia
BaseDeltaDia = dfFinal[dfFinal.DATA == dfFinal.DATA.unique()[-1]].copy()

# Ficheiro Mês
BaseDeltaMes = dfFinal[dfFinal.DATA >= dfFinal.DATA.unique()[-30]].copy()

# Ficheiro 180 dias
BaseDelta180Dias = dfFinal[dfFinal.DATA >= dfFinal.DATA.unique()[-180]].copy()

In [293]:
len(dfFinal.DATA.unique())

180

- Passar para csv

# Clickhouse

In [294]:
import clickhouse_connect
from clickhouse_driver import Client
from unidecode import unidecode

client = clickhouse_connect.get_client(host='ch.brandsandninjas.com', 
                                       port=443, 
                                       username='chninja', 
                                       password='ku43ueqnB5Q0AYb2C4FsJRTc7qX')

In [295]:
BaseDeltaDia.head()

,DATA,DESC_ARTIGO,STORE_NAME,Adequacao,Adequacao_1_Dias_Antes,Adequacao_4_Dias_Antes,Adequacao_5_Dias_Antes,Adequacao_10_Dias_Antes,EAN,STORE,...,Balance_Mediano_5_Dias_Antes,Balance_Mediano_10_Dias_Antes,Balance_Smart,Balance_Smart_5_Dias_Antes,Dias_para_Rotura_Stock,Dias_para_Rotura_Stock_5_Dias_Antes,Dias_Duracao_Linear,Dias_Duracao_Linear_5_Dias_Antes,Vendedor,InterSupply
33870,2023-08-18,CAFÉ SOLÚVEL DELTA FRASCO 100G,CNT MATOSINHOS,,,Stock Suficiente,Stock Suficiente,Stock Suficiente,5.609060e+12,1.0,...,NaN,NaN,NaN,0.509306,NaN,27.730594,8.647059,27.881279,Carla Teixeira,13.0
33871,2023-08-18,CAFÉ SOLÚVEL DELTA FRASCO 100G,CNT AMADORA,,Stock Suficiente,Stock Suficiente,Stock Suficiente,Stock Suficiente,5.609060e+12,2.0,...,NaN,NaN,NaN,0.318507,NaN,28.034188,10.387016,25.914530,Brizida Almeida,NaN
33872,2023-08-18,CAFÉ SOLÚVEL DELTA FRASCO 100G,CNT CASCAIS,,,Stock Insuf c Forn Desadequado,Stock Insuf c Forn Adequado,Stock Insuf c Forn Desadequado,5.609060e+12,3.0,...,NaN,NaN,NaN,0.402564,NaN,7.846154,8.018913,8.307692,Brizida Almeida,NaN
33873,2023-08-18,CAFÉ SOLÚVEL DELTA FRASCO 100G,CNT LEIRIA,,Stock Suficiente,Stock Suficiente,Stock Suficiente,Stock Suficiente,5.609060e+12,5.0,...,NaN,NaN,NaN,0.403828,NaN,68.267771,11.248366,66.752243,Filipa Dias,NaN
33874,2023-08-18,CAFÉ SOLÚVEL DELTA FRASCO 100G,CNT COIMBRASHOPPING,,Stock Suficiente,Stock Suficiente,Stock Suficiente,Stock Suficiente,5.609060e+12,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,9.818182,8.647059,Ana Marques,NaN


In [298]:
#Estabelecer a base a ser lida

baseCH = BaseDeltaDia.copy()
#2023
tabela = "BaseDeltaDia"

In [299]:



###
baseCH['DATA']= pd.to_datetime(baseCH['DATA'], format='%Y-%m-%d')  # Passar para datetime
new_columns = [unidecode(col) for col in baseCH.columns]           # Tirar acentos e afins
baseCH.columns = new_columns                                       # Aplicar alterações da linha anterior
###

# Tipos de dados
data = ["DATA"]
texto = [col for col in baseCH.columns if baseCH[col].dtype == 'object']
inteiros = [col for col in baseCH.columns if baseCH[col].dtype == 'int64' or baseCH[col].dtype == 'int32']
floats = [col for col in baseCH.columns if baseCH[col].dtype == 'float64']


# Só floats é que permitem missing values
for col_name in inteiros:
    baseCH[col_name] = baseCH[col_name].astype(float)
# Missing values em strings estragam tudo
baseCH[texto] = baseCH[texto].fillna("-")


def schema(lista, tipo):
    result_list = [f"{element} {tipo}" for element in lista]
    return result_list

data1 = schema(data, "Date")
texto1 = schema(texto, "String")
inteiros1 = schema(inteiros, "Float64")
floats1 = schema(floats, "Float64")
total = tuple(data1 + texto1 + inteiros1 + floats1)

schema = ', '.join([column.replace("'", "") for column in total])

# Split the input string by commas
parts = schema.split(', ')
# Process each part and wrap the first word in double quotes
output_parts = []
for part in parts:
    words = part.split()
    if words:
        first_word = words[0]
        remaining_words = ' '.join(words[1:])
        output_part = f'"{first_word}" {remaining_words}'
        output_parts.append(output_part)
# Join the modified parts back into a string
schema = ', '.join(output_parts)



# Eliminar tabela no CH
client.command(f'DROP TABLE IF EXISTS {tabela}')

# Criar tabela no CH
client.command(f'''
    CREATE TABLE IF NOT EXISTS {tabela} (
        {schema}
        ) ENGINE = MergeTree
        ORDER BY (DATA)
''')

client.insert_df(tabela, baseCH, column_names=baseCH.columns.tolist())